# Notebook du micro-projet
## But du dataset:
Ce fichier est disponible sur le git : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier récupère et prépare les données pour qu'elles soit exploité par le réseau de neurone.

## Déroulement du code :

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import dataset
import random

Using TensorFlow backend.


Commençons par bon nombre d'import. Principalement lié à Keras et Tensorflow (dont Keras est une couche).
Jusque là  en se qui concerne le code, tout va bien.

On commence alors par aller chercher les différentes images.

In [2]:
def load_data():
    # Load images
    images = np.load('datasets/images.npy')

Ces images sont dans images.npy. Ensuite on met la valeur indiquer par chacune des images. Comme ça on pourra vérifier sir le réseau a bon ou non.

In [3]:
    classes = np.zeros(images.shape[0])
    classes[:204] = 9
    classes[204:409] = 0
    classes[409:615] = 7
    classes[615:822] = 6
    classes[822:1028] = 1
    classes[1028:1236] = 8
    classes[1236:1443] = 4
    classes[1443:1649] = 3
    classes[1649:1855] = 2
    classes[1855:] = 5

NameError: name 'images' is not defined

Ensuite on recadre bien les images pour donné un "norme" pour facilité tous les calculs/comparaison.

In [ ]:
    classes = np_utils.to_categorical(classes, 10)
    images = images.reshape(images.shape[0], 64, 64, 1)

On mélange bien les valeurs qui était jusque là trié.

In [ ]:
    combined = list(zip(images, classes))
    random.shuffle(combined)
    images[:], classes[:] = zip(*combined)

On fini en séparant en 80% des données pour l'entraînement et 20% pour le test.

In [ ]:
    splitIndex = int(len(images) * 0.8)
    x_train = images[:splitIndex]
    x_test = images[splitIndex:]
    y_train = classes[:splitIndex]
    y_test = classes[splitIndex:]
    
    return (x_train, y_train), (x_test, y_test)

## But du modèle :
Ce fichier est disponible sur le git : [GitHub Project language_of_sign_recognition](https://github.com/morvan-s/language_of_sign_recognition)

Ce fichier centralise une grande partie de se projet et en particulier le model qui va réunir les différentes couches du réseau.

## Déroulement du code :

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(64, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(128, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Convolution2D(256, (2, 2), activation='relu', input_shape=(64, 64, 1), data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

**Commentaire sur le model**

Ensuite on compile ce model, pour ce faire on a 3 paramètres, en particulier l'optimizer (adam, optimiseur stochastique) et le loss (crossentropy car ce ne sont pas de simple entier) :

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Il y a ensuite un load. Il appel une méthode de l'autre fichier python (DataSet).

In [ ]:
(x_train, y_train), (x_test, y_test) = dataset.load_data()

Ici nous avons 4 paramètres d'initialiser :
- Epochs : Le nombre d'époque utiliser. Plus cette valeur est élever plus le réseau sera efficace. Mais plus le code sera long à  exécuter.
- Batch_size : La taille des différents lots.
- Data_augmentation : Comme son nom l'indique, ce booléen indique si on utilise la data augmentation (multiplié le nombre de données)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 32
DATA_AUGMENTATION = True

Ensuite, en fonction de la nécessité ou non d'augmenter le nombre données, on va ou non créer un générateur.

In [ ]:
if DATA_AUGMENTATION:
    # With data augmentation
    generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=[0.7, 1.3],
    )
    model.fit_generator(
        generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(x_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=generator.flow(x_test, y_test, batch_size=BATCH_SIZE)
    )
else:
    # Without data augmentation
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(x_test, y_test)
)

Le générateur est un ImageDataGenerator, générateur intégrer à Keras. Ici on a pris quelques décision pour l'augmentation tel que la rotation des images et l'intervalle de zoom. On utilise pour finir le model créer précédemment pour entraîné le réseau.
Pour finir on va tester l'efficacité du réseau de neurone grâce à la méthode evaluate du model.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print('Test loss:', loss)
print('Test accuracy:', accuracy)